In [1]:
from pprint import pprint
import json

name = 'first-batch';

with open('{}.json'.format(name)) as f:
    data = json.load(f)['logs']
    
with open('images/_airlines.json') as f:
    airlines = json.load(f)

with open('images/_states.json') as f:
    states = json.load(f)

In [2]:
from collections import OrderedDict 
from copy import deepcopy

rows = []

for p in data.items():
    user = p[0]
    d = OrderedDict({
        'id': user
    })
    
    item = p[1]
    
    if not 'done' in item:
        print('ignoring {}'.format(p[0]))
        continue
    
    d['userAgent'] = item['userAgent']
    d['isMobile'] = item['isMobile']
    
    cond = item['cond']
    
    errorAirline = cond['errorAirline']
    seqAirline = cond['seqAirline']
    errorStates = cond['errorStates']
    seqStates = cond['seqStates']
    focusAirline = cond['focusAirline']
    focusState = cond['focusState']
    
    for k, v in cond.items():
        d[k] = v
        
    howManyIndex = 0
    for k, v in item['approximateVis_airline'].items():
        t = v['type']
        
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'approx.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
    
    howManyIndex = 0
    for k, v in item['approximateVis_states'].items():
        t = v['type']
        
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'approx.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
     
    def jaccard_similarity(x,y):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    
    # precise
            
    howManyIndex = 0
    for k, v in item['preciseVis_airline'].items():
        t = v['type']
        
        precise = airlines["{}_{}".format(1, 1)]
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'precise.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'precise'] = precise[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorAirline > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            d[p + 'jaccard_approx_precise'] = jaccard_similarity(appr, prec)
            d[p + 'jaccard_approx_answer'] = jaccard_similarity(appr, answers)
            
    howManyIndex = 0
    for k, v in item['preciseVis_states'].items():
        t = v['type']
        
        precise = states["{}_{}".format(1, 1)]
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'precise.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'precise'] = precise[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorStates > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            d[p + 'jaccard_approx_precise'] = jaccard_similarity(appr, prec)
            d[p + 'jaccard_approx_answer'] = jaccard_similarity(appr, answers)
    
    for k, v in item['timing'].items():
         d['timing.' + k] = v
            
    for k, v in item['demographics'].items():
         d['demographics.' + k] = v
    
    rows.append(d)

pprint(rows[0])

ignoring 41061051
ignoring 13623650
OrderedDict([('id', '66972706'),
             ('userAgent',
              'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
              '(KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'),
             ('isMobile', False),
             ('secondCondition', 'states'),
             ('focusState', 'MA'),
             ('firstCondition', 'airline'),
             ('seqAirline', 15),
             ('errorStates', 100),
             ('seqStates', 5),
             ('focusAirline', 'DL'),
             ('studyCondition', 'both'),
             ('errorAirline', 500),
             ('approx.airline.howManyCompare.0.answer', 3000),
             ('approx.airline.howManyCompare.0.data', 'UA'),
             ('approx.airline.howManyCompare.0.approx', 1000),
             ('approx.airline.howManyCompare.0.confidence', 'agree'),
             ('approx.airline.howManyCompare.1.answer', 4000),
             ('approx.airline.howManyCompare.1.data', 'AA'),
   

In [3]:
import csv

with open('{}.csv'.format(name), 'w') as f:
    fieldnames = rows[1].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in rows:
        writer.writerow(row)

import pandas as pd

pd.read_csv('{}.csv'.format(name))

,id,userAgent,isMobile,secondCondition,focusState,firstCondition,seqAirline,errorStates,seqStates,focusAirline,...,timing.index_html,timing.precise_instructions_html,timing.thanks_html,demographics.improved,demographics.age,demographics.familiar,demographics.troubles,demographics.cheat,demographics.gender,demographics.problems
0,66972706,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,states,MA,airline,15,100,5,DL,...,11056,2711,14204,-,27,familiar,no,no,Female,no
1,18074862,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,airline,NY,states,1,500,13,DL,...,1407,8558,13489,-,26,familiar,no,no,male,-
2,58926040,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,False,states,MA,airline,3,1000,3,B6,...,1949,3239,209269,Everything was clear!,30,familiar,no,no,Male,None at all!
3,42878404,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,False,airline,NY,states,8,100,12,B6,...,5233,4809,8623,-,36,familiar,no,no,male,no issues
4,84835951,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,states,NY,airline,19,200,14,AA,...,330079,7292,60642,no,27,familiar,no,no,male,-
5,49437914,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4)...,False,states,FL,airline,1,1,1,DL,...,5901,6908,1866427,it was all good!,22,familiar,no,no,female,no
6,55656520,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5...,False,airline,MA,states,20,500,19,B6,...,2482,3250,5809,-,33,some,no,no,female,-
7,80993444,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,False,states,NY,airline,4,1,1,AA,...,2785,5823,22486,No.,34,some,no,no,male,I didn't have any problems.
8,55686356,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,False,states,FL,airline,17,200,6,DL,...,977,2577,2801,-,31,some,yes,no,female,no
9,23634915,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,states,MA,airline,12,1,1,AA,...,49309,5655,18522,No everything was clear and easy to understand...,24,some,no,no,female,no
